In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [6]:
!pip install hickle
!pip install segyio
!pip install -q keras

In [ ]:
import os
import hickle
import json
import random
import shutil
import segyio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from sklearn.feature_extraction import image
from keras import backend as K
from keras.utils.np_utils import to_categorical
import keras
import numpy as np
from skimage.measure import regionprops, label
import matplotlib.patches as mpatches
from sklearn.utils import shuffle as shu
from keras import metrics
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.utils import conv_utils
from keras.utils.data_utils import get_file
import random as rand
from keras.layers import LSTM
from keras import utils as np_utils
from skimage.util.shape import view_as_windows
from sklearn.utils import shuffle
from skimage import exposure
from sklearn.metrics import roc_curve, auc, multilabel_confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from keras.layers.noise import GaussianNoise
from keras.layers.advanced_activations import LeakyReLU
from keras.initializers import TruncatedNormal
from keras.utils.generic_utils import Progbar
# from keras.utils import multi_gpu_model
from sklearn.metrics import average_precision_score
from tensorflow.python.ops import control_flow_ops
import tensorflow as tf

# from keras.layers import CuDNNLSTM
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report, accuracy_score
K.set_image_data_format('channels_last')
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
def buildPatches(brain, wmlMask, mode='train'):

    if (mode=='train'):
        
        patches = view_as_windows(brain, (64,64), step=(16,32))
        labelPatch = view_as_windows(wmlMask, (64,64), step=(16,32))
        patchesBrain = []
        patchesWml = []
        
        for i in range(patches.shape[0]):
            
            for j in range(patches.shape[1]):
                    if(labelPatch[i][j].any()):
                        patchesBrain.append(patches[i][j])
                        patchesWml.append(labelPatch[i][j])
            
 
        return np.array(patchesBrain), np.array(patchesWml)

    else:
        # Extract patches.
        #Validation and Test doesnt perform trace balancing step.
        patches = view_as_windows(brain, (64,64), step=(16,32))
        labelPatch = view_as_windows(wmlMask, (64,64), step=(16,32))
        patchesBrain = []
        patchesWml = []
        
        for i in range(patches.shape[0]):
            
            for j in range(patches.shape[1]):
                    patchesBrain.append(patches[i][j])
                    patchesWml.append(labelPatch[i][j])
                    
        return np.array(patchesBrain), np.array(patchesWml)


In [ ]:
# trainSamplesInline = os.listdir('../content/gdrive/MyDrive/tcc/numpy/train/')
# validationSamplesInline = os.listdir('../content/gdrive/MyDrive/tcc/numpy/validation/')
# testSamplesInline = os.listdir('../content/gdrive/MyDrive/tcc/numpy/test/')

patchesTrainInline = []
patchesValidationInline = []
patchesTestInline = []

labelsTrainInline = []
labelsValidationInline = []
labelsTestInline = []

In [ ]:
samplesInline = os.listdir('../content/gdrive/MyDrive/tcc/numpy/dados/')
labelsInline = os.listdir('../content/gdrive/MyDrive/tcc/numpy/label/')

inlineSamples = []
inlineLabels = []

In [ ]:
for file in samplesInline:
    brain = np.load(f'../content/gdrive/MyDrive/tcc/numpy/dados/{file}')
    wmlMask = np.load(f'../content/gdrive/MyDrive/tcc/numpy/label/{file}')
    
    inlineSamples.append(brain)
    inlineLabels.append(wmlMask)

In [ ]:
from sklearn.model_selection import train_test_split
train_ratio = 0.82
validation_ratio = 0.10
test_ratio = 0.08

trainPatchNumpy, testPatchNumpy, trainLabelNumpy, testLabelNumpy = train_test_split(inlineSamples, inlineLabels, test_size=1 - train_ratio, random_state=11)

validationPatchNumpy, testPatchNumpy, validationLabelNumpy, testLabelNumpy = train_test_split(testPatchNumpy, testLabelNumpy, test_size=test_ratio/(test_ratio + validation_ratio), random_state=11) 

In [ ]:
for i in range(len(trainPatchNumpy)):
  patchesBrainTrain, labelsTrain = buildPatches(trainPatchNumpy[i], trainLabelNumpy[i])
  patchesTrainInline.append(patchesBrainTrain)
  labelsTrainInline.append(labelsTrain)

In [ ]:
for i in range(len(validationPatchNumpy)):
  patchesBrainTrain, labelsTrain = buildPatches(validationPatchNumpy[i], validationLabelNumpy[i], mode='validation')
  patchesValidationInline.append(patchesBrainTrain)
  labelsValidationInline.append(labelsTrain)

In [ ]:
for i in range(len(testPatchNumpy)):
  patchesBrainTrain, labelsTrain = buildPatches(testPatchNumpy[i], testLabelNumpy[i], mode='test')
  patchesTestInline.append(patchesBrainTrain)
  labelsTestInline.append(labelsTrain)

In [ ]:
# for file in trainSamplesInline:
#     brain = np.load(f'../content/gdrive/MyDrive/tcc/numpy/train/{file}')
#     wmlMask = np.load(f'../content/gdrive/MyDrive/tcc/numpy/label/{file}')
    

#     patchesBrainTrain, labelsTrain = buildPatches(brain, wmlMask)

#     patchesTrainInline.append(patchesBrainTrain)
#     labelsTrainInline.append(labelsTrain)

In [ ]:
# for file in trainSamplesInline:
#     brain = np.load(f'../content/gdrive/MyDrive/tcc/numpy/train/{file}')
#     wmlMask = np.load(f'../content/gdrive/MyDrive/tcc/numpy/label/{file}')
    

#     patchesBrainTrain, labelsTrain = buildPatches(brain, wmlMask)

#     patchesTrainInline.append(patchesBrainTrain)
#     labelsTrainInline.append(labelsTrain)

In [ ]:
# for file in validationSamplesInline:
#     brain = np.load(f'../content/gdrive/MyDrive/tcc/numpy/validation/{file}')
#     wmlMask = np.load(f'../content/gdrive/MyDrive/tcc/numpy/label/{file}')
    

#     patchesBrainValidation, labelsValidation = buildPatches(brain, wmlMask, mode="validation")

#     patchesValidationInline.append(patchesBrainValidation)
#     labelsValidationInline.append(labelsValidation)

In [ ]:
# for file in testSamplesInline:
#     brain = np.load(f'../content/gdrive/MyDrive/tcc/numpy/test/{file}')
#     wmlMask = np.load(f'../content/gdrive/MyDrive/tcc/numpy/label/{file}')
    

#     patchesBrainTest, labelsTest = buildPatches(brain, wmlMask, mode="test")

#     patchesTestInline.append(patchesBrainTest)
#     labelsTestInline.append(labelsTest)

In [ ]:
patchesTrainInline = np.concatenate([i for i in patchesTrainInline if i.size > 0])
labelsTrainInline = np.concatenate([i for i in labelsTrainInline if i.size > 0])

In [ ]:
patchesValidationInline = np.concatenate([i for i in patchesValidationInline])
labelsValidationInline = np.concatenate([i for i in labelsValidationInline])

In [ ]:
patchesTestInline = np.concatenate([i for i in patchesTestInline])
labelsTestInline = np.concatenate([i for i in labelsTestInline])

In [ ]:
def unet(pretrained_weights = None,input_size = (64,64, 1)):
    inputs = Input(input_size)
    conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = [inputs], outputs = conv10)



    return model

In [ ]:
early_stop = EarlyStopping(monitor='val_recall_m', mode='max', patience=6)
batch_size = 32

In [ ]:
checkpoint_path = "../content/gdrive/MyDrive/tcc/checkpoints/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=13196)

In [ ]:
model = unet()

latest = tf.train.latest_checkpoint(checkpoint_dir)
epoch = 0
if(latest):
  model.load_weights(latest)
  epoch = int(latest.split('-')[1].strip('0').strip(".ckpt"))
  print(epoch)

41


In [ ]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
def specificity_m(y_true, y_pred):
    """
    param:
    y_pred - Predicted labels
    y_true - True labels 
    Returns:
    Specificity score
    """
    neg_y_true = 1 - y_true
    neg_y_pred = 1 - y_pred
    fp = K.sum(neg_y_true * y_pred)
    tn = K.sum(neg_y_true * neg_y_pred)
    specificity = tn / (tn + fp + K.epsilon())
    return specificity 

def f1Score(y_true, y_pred):

    p = precision_m(y_true, y_pred)
    r = recall_m(y_true, y_pred)

    f1 = 2*p*r / (p+r+K.epsilon())

    return f1
    
def auc_roc(y_true, y_pred):
    value, update_op = tf.contrib.metrics.streaming_auc(
        y_pred, y_true, curve='ROC', name='auc_roc')
    auc_roc.stream_vars = [i for i in tf.local_variables() if i.name.split('/')[0] == 'auc_roc']
    return control_flow_ops.with_dependencies([update_op], value)

In [ ]:
patchesTrainInline = patchesTrainInline.reshape( patchesTrainInline.shape[0], 64, 64, 1)  / patchesTrainInline.max()
labelsTrainInline = labelsTrainInline.reshape( labelsTrainInline.shape[0], 64, 64, 1)
patchesTrainInline = patchesTrainInline.astype( 'float64' )

patchesValidationInline = patchesValidationInline.reshape(patchesValidationInline.shape[0], 64, 64, 1) / patchesValidationInline.max()
labelsValidationInline = labelsValidationInline.reshape(labelsValidationInline.shape[0], 64, 64, 1)
patchesValidationInline = patchesValidationInline.astype( 'float64' )

# # ### Test
patchesTestInline = patchesTestInline.reshape(patchesTestInline.shape[0], 64, 64, 1) / patchesTestInline.max()
labelsTestInline = labelsTestInline.reshape(labelsTestInline.shape[0], 64, 64, 1)
patchesTestInline = patchesTestInline.astype( 'float64' )

In [ ]:
import gc
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import Callback


class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        k.clear_session()

In [ ]:
# from sklearn.utils import class_weight

# weights = class_weight.compute_class_weight('balanced',
#                                             np.unique(np.ravel(labelsTrainInline,order='C')),
#                                             np.ravel(labelsTrainInline,order='C'))

# labelsTrainInline = to_categorical(labelsTrainInline)
# labelsValidationInline = to_categorical(labelsValidationInline)

In [ ]:
# def weighted_categorical_crossentropy(weights):
#         """
#         A weighted version of keras.objectives.categorical_crossentropy
#         Variables:
#             weights: numpy array of shape (C,) where C is the number of classes
#         Usage:
#             weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
#             loss = weighted_categorical_crossentropy(weights)
#             model.compile(loss=loss,optimizer='adam')
#         """
#         weights = K.variable(weights)
#         def loss(y_true, y_pred):
#             # scale predictions so that the class probas of each sample sum to 1
#             y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
#             # clip to prevent NaN's and Inf's
#             y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
#             loss = y_true * K.log(y_pred) + (1-y_true) * K.log(1-y_pred)
#             loss = loss * weights 
#             loss = - K.mean(loss, -1)
#             return loss
#         return loss
# 
# loss_function = weighted_categorical_crossentropy(weights)

In [ ]:
param = dict(
  rotation_range=30,
	# zoom_range=0.15,
	# width_shift_range=0.2,
	# height_shift_range=0.2,
	# shear_range=0.15,
	horizontal_flip=True,
  vertical_flip=True,
  # featurewise_center=True,
  # samplewise_center=True,
  # featurewise_std_normalization=True,
  # samplewise_std_normalization=True,
  # zca_whitening=True,
  data_format="channels_last",
  dtype="float64",
)
 
generator = ImageDataGenerator(**param)

gc.collect()

input = generator.flow(patchesTrainInline, batch_size=batch_size*2, seed=2)
mask = generator.flow(labelsTrainInline, batch_size=batch_size*2, seed=2)

validation_input = generator.flow(labelsTrainInline, batch_size=batch_size, seed=2)
validation_mask = generator.flow(labelsValidationInline, batch_size=batch_size, seed=2)

del patchesTrainInline
del patchesValidationInline
del labelsTrainInline
del labelsValidationInline
del generator
gc.collect()

train_generator = zip(input, mask)
validation_generator = zip(validation_input, validation_mask)

del input
del mask
del validation_input
del validation_mask
gc.collect()

0

In [1]:
# combinedPatches = np.concatenate((patchesTrainInline, patchesValidationInline), axis=0)
# combinedLabels = np.concatenate((labelsTrainInline, labelsValidationInline), axis=0)
 
# newTrainPatch, newValidationPatch, newTrainLabel, newValidationLabel = train_test_split(combinedPatches, combinedLabels, test_size=0.1, random_state=11)

# input = generator.flow(newTrainPatch, batch_size=batch_size*2, seed=2)
# mask = generator.flow(newTrainLabel, batch_size=batch_size*2, seed=2)

# validation_input = generator.flow(newValidationPatch, batch_size=batch_size, seed=2)
# validation_mask = generator.flow(newValidationLabel, batch_size=batch_size, seed=2)

# combinedGenerator = zip(input, mask)
# combinedValidationGenerator = zip(validation_input, validation_mask)

In [4]:
model.compile(optimizer=keras.optimizers.Adadelta(), loss= "binary_crossentropy", metrics = ['accuracy', recall_m, specificity_m, precision_m, f1Score])
model.summary()

In [3]:
history = model.fit(x=train_generator, initial_epoch=epoch, verbose=1, batch_size=batch_size, steps_per_epoch=len(newTrainPatch) / (batch_size / 8), epochs=100, callbacks=[ClearMemory(), early_stop, cp_callback],
                   validation_data=validation_generator, validation_steps=batch_size)

In [ ]:
# history = model.fit(x=combinedPatches, y=combinedLabels, initial_epoch=epoch, verbose=1, batch_size=batch_size, steps_per_epoch=len(combinedPatches) / (batch_size / 8), epochs=100, callbacks=[ClearMemory(), early_stop, cp_callback],
#                    validation_split=0.1)

In [2]:
results = model.evaluate(patchesTestInline, labelsTestInline, batch_size=32)
results

In [ ]:
# preds_val = model.predict(patchesValidationInline, verbose=1)

In [ ]:
def predictSection(input, step=(16,32)):
    patches = view_as_windows(input, (64,64), step=(16,32))
    vectorPatches = []
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
                    vectorPatches.append(patches[i][j])
                
    vectorPatches = np.array(vectorPatches)
    vectorPatches = vectorPatches.reshape( vectorPatches.shape[0], 64, 64, 1)  / vectorPatches.max()
    section = reconstruct_from_patches_2d(vectorPatches, (384, 384, 1), stride=(16, 32))  
    
    prediction = model.predict(vectorPatches, verbose=1)
    prediction[prediction > 0.5] = 1
    prediction[prediction <= 0.5] = 0
    # predictionSector = reconstruct_from_patches_2d(np.argmax(prediction, axis=-1), (384, 384), stride=(16, 32))
    predictionSector = reconstruct_from_patches_2d(prediction, (384, 384, 1), stride=(16, 32))
    
    
    return input, predictionSector, prediction

In [ ]:
def reconstruct_from_patches_2d(patches,image_shape,stride):
  img = np.zeros(image_shape,dtype=np.float64)
  img_repeticoes = np.zeros(image_shape)
  patch_shape = patches.shape[1:3]
  i=0
  for x in range(0,image_shape[0],stride[0]):
    for y in range(0,image_shape[1],stride[1]):
      if image_shape[0] >= x+patch_shape[0] and image_shape[1]>=y+patch_shape[1] :
        img[x:x+patch_shape[0],y:y+patch_shape[1]]+=patches[i]
        img_repeticoes[x:x+patch_shape[0],y:y+patch_shape[1]]+=np.ones(patches.shape[1:])
        i+=1
  img = img//img_repeticoes
  return img

In [5]:
for i in range(len(testLabelNumpy)):
  patch = testPatchNumpy[i]
  mask = testLabelNumpy[i]
  plt.figure(figsize=(10,10))
  plt.subplot(2, 2, 1)
  plt.imshow(patch, cmap="gray")
  plt.imshow(mask, cmap='jet', alpha=0.5)
  plt.axis('off')

  section, prediction, pred  = predictSection(patch)
  plt.subplot(2, 2, 2)
  plt.imshow(section.reshape(384, 384), cmap="gray")
  plt.imshow(prediction.reshape(384, 384), cmap="jet", alpha=0.5)
  plt.axis('off')
  plt.show()